### Mindemo

In [ ]:
#### Required libraries ####
# !pip install numpy
# !pip install matplotlib

#### Import ####
import copy
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
## Functions for manipulating tensors
def get_N(N_size):
    ## When transforming a tensor to ndarray with np.reshape, use order="F", for example, np.reshape(a=target_tensor, newshape=N_size, order="F")
    if len(N_size) < 1:
        raise ValueError("Error: N_size is invalid.")
    N_rank = len(N_size)
    N_accum = np.ones(N_rank, dtype=int) ## n2*n3*...*nN, n3*n4*...*nN, ... , nN, 1
    for i in range(N_rank):
        if i==0:
            N_accum[N_rank-i-1] = 1
        else:
            N_accum[N_rank-i-1] = N_accum[N_rank-i]*N_size[N_rank-i]
    N_size_prod = N_size[0]*N_accum[0]
    return (N_rank, N_accum, N_size_prod)

def get_tensor_flattened_index_from_multi_index(multi_index, N_rank, N_accum):
    ## When transforming a tensor to ndarray with np.reshape, use order="F", for example, np.reshape(a=target_tensor, newshape=N_size, order="F")
    flattened_index = 0
    for i in range(N_rank):
            flattened_index = flattened_index + N_accum[i]*multi_index[i]
    flattened_index = int(flattened_index)
    return flattened_index

def get_tensor_multi_index_from_flattened_index(flattened_index, N_rank, N_accum):
    ## When transforming a tensor to ndarray with np.reshape, use order="F", for example, np.reshape(a=target_tensor, newshape=N_size, order="F")
    multi_index = []
    remainder = flattened_index
    for i in range(N_rank):
        quotient, remainder = divmod(remainder, N_accum[i])
        multi_index.append(quotient)
    multi_index = tuple(multi_index)
    return multi_index

def get_tensor_value_from_multi_index(target_tensor, multi_index, N_rank, N_accum):
    ## When transforming a tensor to ndarray with np.reshape, use order="F", for example, np.reshape(a=target_tensor, newshape=N_size, order="F")
    flattened_index = get_tensor_flattened_index_from_multi_index(multi_index, N_rank, N_accum)
    return target_tensor[flattened_index]

def get_tensor_flattened_index_list_from_value(target_tensor, value, tensor_tolerance=None):
    ## When transforming a tensor to ndarray with np.reshape, use order="F", for example, np.reshape(a=target_tensor, newshape=N_size, order="F")
    if (tensor_tolerance is None) or (tensor_tolerance==0):
        return [i for i, element in enumerate(target_tensor) if element==value]
    else:
        return [i for i, element in enumerate(target_tensor) if abs(element-value)<=tensor_tolerance]

def get_tensor_multi_index_list_from_value(target_tensor, value, N_rank, N_accum, tensor_tolerance=None):
    ## When transforming a tensor to ndarray with np.reshape, use order="F", for example, np.reshape(a=target_tensor, newshape=N_size, order="F")
    multi_index_list = []
    flattened_index_list = get_tensor_flattened_index_list_from_value(target_tensor, value, tensor_tolerance)
    for flattened_index in flattened_index_list:
        multi_index_list.append(get_tensor_multi_index_from_flattened_index(flattened_index, N_rank, N_accum))
    return multi_index_list

## Functions to calculate costs of optrimal transport, etc.
# @jit
def calc_multi_ot(marginal_mass_vectors, cost_tensor, normalized_cost_tensor,
                  N_size, N_rank, N_accum, N_size_prod,
                  numerical_precision=2e-8, ot_speed=0.02, ot_stopping_rule=0.02, ot_loop_max=200): ## ot_stopping_rule: Criteria to stop updating "u". If the relative error of "u" is smaller than the stop criterion, it is terminated.
    ## Optrimal transport
    K_tensor = np.exp(- normalized_cost_tensor / ot_speed) ## Gibbs kernel
    u_vec_list = []
    for i in range(N_rank):
        u_vec_list.append(np.ones(N_size[i]))
    for loop in range(ot_loop_max):
        u_diff = 0 ## Variable to measure whether to exit the loop
        for i in range(N_rank):
            for j in range(N_size[i]):
                temp_u_value = 0
                temp_K_value = 1
                N_size_sub = list(copy.copy(N_size))
                N_size_sub.pop(i)
                for m_sub_index in np.ndindex(tuple(N_size_sub)):
                    temp_m_index = list(copy.copy(m_sub_index))
                    temp_m_index.insert(i, j)
                    temp_K_value = get_tensor_value_from_multi_index(K_tensor, temp_m_index, N_rank, N_accum)
                    temp_u_prod_value = 1
                    for k in range(N_rank):
                        if k != i:
                            temp_u_prod_value = temp_u_prod_value * u_vec_list[k][temp_m_index[k]]
                    temp_u_value = temp_u_value + temp_K_value * temp_u_prod_value
                temp_u_value = (marginal_mass_vectors[i][j]) / (temp_u_value)
                u_diff = max(u_diff, abs((u_vec_list[i][j]-temp_u_value)/(temp_u_value+numerical_precision))) 
                u_vec_list[i][j] = temp_u_value
        if abs(u_diff) < ot_stopping_rule:
            break
    f_vec_list = []
    for i in range(N_rank):
        temp_f_vec = ot_speed * np.log(u_vec_list[i] + numerical_precision)
        f_vec_list.append(temp_f_vec)
    P_tensor = np.zeros(N_size_prod)
    weighted_cost_tensor = np.zeros(N_size_prod)
    objective_function_value = 0
    for m_index in np.ndindex(tuple(N_size)):
        temp_cost_value = get_tensor_value_from_multi_index(cost_tensor, m_index, N_rank, N_accum)
        temp_P_value = get_tensor_value_from_multi_index(K_tensor, m_index, N_rank, N_accum)
        for k in range(N_rank):
            temp_P_value = temp_P_value * u_vec_list[k][m_index[k]]
        P_tensor[get_tensor_flattened_index_from_multi_index(m_index, N_rank, N_accum)] = temp_P_value
        weighted_cost_tensor[get_tensor_flattened_index_from_multi_index(m_index, N_rank, N_accum)] = temp_P_value*temp_cost_value
        objective_function_value = objective_function_value + weighted_cost_tensor[get_tensor_flattened_index_from_multi_index(m_index, N_rank, N_accum)]
    return (objective_function_value, P_tensor, weighted_cost_tensor, u_vec_list, f_vec_list)

## Functions for Mindemo
def gen_marginal_density_ndarray_for_mindemo(x_size, marginal_x_density_type="Uniform",
                                             x_ndarray=None, marginal_x_params_dict=None,
                                             numerical_precision=2e-8):
    density_ndarray = np.ones(x_size)
    if marginal_x_density_type == "Normal":
        x_mu = np.mean(x_ndarray) + marginal_x_params_dict.get("location",0)
        x_sig = np.std(x_ndarray)
        if x_sig > 0:
            if marginal_x_params_dict.get("scale",0) > 0:
                x_sig = x_sig * marginal_x_params_dict.get("scale",0)
            for i in range(x_size):
                x_value = x_ndarray[i]
                density_ndarray[i] = np.exp( -(x_value - x_mu)**2 / (2*(x_sig)**2) ) / ( np.sqrt(2*np.pi) * x_sig )
    elif marginal_x_density_type == "Exp":
        for i in range(x_size):
            x_value = x_ndarray[i]
            density_ndarray[i] = 0
            density_ndarray[i] = marginal_x_params_dict.get("@*1",0)
            density_ndarray[i] = density_ndarray[i] + marginal_x_params_dict.get("@*x",0) * x_value
            density_ndarray[i] = density_ndarray[i] + marginal_x_params_dict.get("@*xx",0) * x_value * x_value
            density_ndarray[i] = density_ndarray[i] + marginal_x_params_dict.get("@*xxx",0) * x_value * x_value * x_value
            density_ndarray[i] = density_ndarray[i] + marginal_x_params_dict.get("@*xxxx",0) * x_value * x_value * x_value * x_value
            density_ndarray[i] = density_ndarray[i] + marginal_x_params_dict.get("@*abs(1+x+xx)",0) * np.abs(marginal_x_params_dict.get("abs(@*1+x+xx)",0) +
                                                                                    marginal_x_params_dict.get("abs(1+@*x+xx)",0) * x_value +
                                                                                    marginal_x_params_dict.get("abs(1+x+@*xx)",0) * x_value * x_value)
            density_ndarray[i] = density_ndarray[i] + marginal_x_params_dict.get("@*sin(1+x+xx)",0) * np.sin(2*np.pi*(marginal_x_params_dict.get("sin(@*1+x+xx)",0) +
                                                                                    marginal_x_params_dict.get("sin(1+@*x+xx)",0) * x_value +
                                                                                    marginal_x_params_dict.get("sin(1+x+@*xx)",0) * x_value * x_value))
            density_ndarray[i] = density_ndarray[i] + marginal_x_params_dict.get("@*cos(1+x+xx)",0) * np.cos(2*np.pi*(marginal_x_params_dict.get("cos(@*1+x+xx)",0) +
                                                                                    marginal_x_params_dict.get("cos(1+@*x+xx)",0) * x_value +
                                                                                    marginal_x_params_dict.get("cos(1+x+@*xx)",0) * x_value * x_value))
            density_ndarray[i] = np.exp(density_ndarray[i])
    else: ## Uniform
        if marginal_x_params_dict.get("scale",0) > 0:
            x_min = marginal_x_params_dict.get("location",0) - marginal_x_params_dict.get("scale",0)
            x_max = marginal_x_params_dict.get("location",0) + marginal_x_params_dict.get("scale",0)
            for i in range(x_size):
                x_value = x_ndarray[i]
                if x_value < (x_min - numerical_precision):
                    density_ndarray[i] = 0
                elif x_value > (x_max + numerical_precision):
                    density_ndarray[i] = 0
            if np.sum(density_ndarray) <= 0:
                density_ndarray = np.ones(x_size)
    density_ndarray = density_ndarray / sum(density_ndarray)
    return density_ndarray

def gen_marginal_densities_ndarray_list_for_mindemo(N_rank, N_size, xyz_ndarray_list,
                                                  marginal_density_types_list, marginal_params_dict_list,
                                                  numerical_precision=2e-8):
    marginal_density_ndarray_list = []
    for i in range(N_rank):
        marginal_density_ndarray_list.append(gen_marginal_density_ndarray_for_mindemo(N_size[i], marginal_density_types_list[i], xyz_ndarray_list[i], marginal_params_dict_list[i], numerical_precision))
    return marginal_density_ndarray_list

def gen_cost_value_for_mindemo(x, y, mindemo_params_dict):
    model = 0
    model = mindemo_params_dict.get("@*1",0)
    model = model + mindemo_params_dict.get("@*x",0) * x
    model = model + mindemo_params_dict.get("@*y",0) * y
    model = model + mindemo_params_dict.get("@*xx",0) * x * x
    model = model + mindemo_params_dict.get("@*xy",0) * x * y
    model = model + mindemo_params_dict.get("@*yy",0) * y * y
    model = model + mindemo_params_dict.get("@*xxx",0) * x * x * x
    model = model + mindemo_params_dict.get("@*xxy",0) * x * x * y
    model = model + mindemo_params_dict.get("@*xyy",0) * x * y * y
    model = model + mindemo_params_dict.get("@*yyy",0) * y * y * y
    model = model + mindemo_params_dict.get("@*xxxx",0) * x * x * x * x
    model = model + mindemo_params_dict.get("@*xxxy",0) * x * x * x * y
    model = model + mindemo_params_dict.get("@*xxyy",0) * x * x * y * y
    model = model + mindemo_params_dict.get("@*xyyy",0) * x * y * y * y
    model = model + mindemo_params_dict.get("@*yyyy",0) * y * y * y * y
    model = model + mindemo_params_dict.get("@*abs(1+x+y+xx+xy+yy)",0) * np.abs(mindemo_params_dict.get("abs(@*1+x+y+xx+xy+yy)",0) +
                                                                              mindemo_params_dict.get("abs(1+@*x+y+xx+xy+yy)",0)*x +
                                                                              mindemo_params_dict.get("abs(1+x+@*y+xx+xy+yy)",0)*y +
                                                                              mindemo_params_dict.get("abs(1+x+y+@*xx+xy+yy)",0)*x*x +
                                                                              mindemo_params_dict.get("abs(1+x+y+xx+@*xy+yy)",0)*x*y +
                                                                              mindemo_params_dict.get("abs(1+x+y+xx+xy+@*yy)",0)*y*y)
    model = model + mindemo_params_dict.get("@*sin(1+x+y+xx+xy+yy)",0) * np.sin(2*np.pi*(mindemo_params_dict.get("sin(@*1+x+y+xx+xy+yy)",0) +
                                                                                     mindemo_params_dict.get("sin(1+@*x+y+xx+xy+yy)",0)*x +
                                                                                     mindemo_params_dict.get("sin(1+x+@*y+xx+xy+yy)",0)*y +
                                                                                     mindemo_params_dict.get("sin(1+x+y+@*xx+xy+yy)",0)*x*x +
                                                                                     mindemo_params_dict.get("sin(1+x+y+xx+@*xy+yy)",0)*x*y +
                                                                                     mindemo_params_dict.get("sin(1+x+y+xx+xy+@*yy)",0)*y*y))
    model = model + mindemo_params_dict.get("@*cos(1+x+y+xx+xy+yy)",0) * np.cos(2*np.pi*(mindemo_params_dict.get("cos(@*1+x+y+xx+xy+yy)",0) +
                                                                                     mindemo_params_dict.get("cos(1+@*x+y+xx+xy+yy)",0)*x +
                                                                                     mindemo_params_dict.get("cos(1+x+@*y+xx+xy+yy)",0)*y +
                                                                                     mindemo_params_dict.get("cos(1+x+y+@*xx+xy+yy)",0)*x*x +
                                                                                     mindemo_params_dict.get("cos(1+x+y+xx+@*xy+yy)",0)*x*y +
                                                                                     mindemo_params_dict.get("cos(1+x+y+xx+xy+@*yy)",0)*y*y))
    model = (-1) * model
    return model

def gen_cost_tensor_for_mindemo(xyz_ndarray_list, 
                     N_size, N_rank, N_accum, N_size_prod,
                     mindemo_params_dict,
                     numerical_precision=2e-8):
    cost_tensor = np.zeros(N_size_prod)
    for m_index in np.ndindex(N_size):
        xyz_values_list = [xyz_ndarray_list[var][i] for var, i in enumerate(m_index)]
        temp_index = get_tensor_flattened_index_from_multi_index(m_index, N_rank, N_accum)
        cost_tensor[temp_index] = gen_cost_value_for_mindemo(xyz_values_list[0], xyz_values_list[1], mindemo_params_dict)
    max_cost_value = max(cost_tensor)
    normalized_cost_tensor = copy.deepcopy(cost_tensor)
    if max_cost_value > numerical_precision:
        normalized_cost_tensor = normalized_cost_tensor/max_cost_value
    return (cost_tensor, normalized_cost_tensor)

def gen_density_for_mindemo(xyz_ndarray_list, marginal_density_ndarray_list,
                     N_size, N_rank, N_accum, N_size_prod, 
                     mindemo_params_dict, numerical_precision=2e-8):
    ## cost tensor
    (cost_tensor, normalized_cost_tensor) = gen_cost_tensor_for_mindemo(xyz_ndarray_list, 
                     N_size, N_rank, N_accum, N_size_prod, 
                     mindemo_params_dict, numerical_precision)
    ## optimal transport ( P_tensor = density in mindemo )
    (objective_function_value, P_tensor, weighted_cost_tensor,
    u_vec_list, f_vec_list) = calc_multi_ot(marginal_density_ndarray_list, cost_tensor, cost_tensor, ## normalized_cost_tensor -(mindemo)-> cost_tensor
                                            N_size, N_rank, N_accum, N_size_prod,
                                            numerical_precision,
                                            ot_speed=1.0, ## ot_speed=0.02 -(mindemo)-> ot_speed=1.0
                                            ot_stopping_rule=0.02, ot_loop_max=200)
    return (cost_tensor, normalized_cost_tensor, objective_function_value, P_tensor, weighted_cost_tensor, u_vec_list, f_vec_list)

def gen_heatmap(target_tensor, xy_ndarray_list, N_size, show=False, fsize=(8.0, 6.0)):
    target_max = max(target_tensor)
    target_array = np.reshape(a=target_tensor, newshape=N_size, order="F")
    fig, ax = plt.subplots(figsize=fsize)
    ax.cla()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ext = (min(xy_ndarray_list[0]), max(xy_ndarray_list[0]), min(xy_ndarray_list[1]), max(xy_ndarray_list[1]))
    fig.set_facecolor("#B0B0B0")
    mappable = ax.imshow(target_array, interpolation='nearest', origin='lower', extent=ext, vmin=0, vmax=target_max, cmap='CMRmap') ## Greens, Blues, GnBu, CMRmap, Spectral
    fig.colorbar(mappable)
    if show:
        plt.show()
    return (fig, ax)

def gen_marginal_density_line_chart(x_ndarray, marginal_density_ndarray, show=False, fsize=(4.0, 3.0)):
    fig, ax = plt.subplots(figsize=fsize)
    ax.cla()
    fig.set_facecolor("#B0B0B0")
    ax.fill_between(x_ndarray, marginal_density_ndarray, color="lightblue", alpha=0.5)
    ax.plot(x_ndarray, marginal_density_ndarray, color="royalblue", alpha=0.5)
    if show:
        plt.show()
    return (fig, ax)


In [ ]:
#### Example ####

numerical_precision = 2e-8
## variables
xyz_step = 0.05
xyz_min = -1
xyz_max = 1
xyz_max_plus = xyz_max + xyz_step
x = np.arange(xyz_min, xyz_max_plus, xyz_step)
y = np.arange(xyz_min, xyz_max_plus, xyz_step)
xyz_ndarray_list = [x, y]
## tensor size
N_size = (len(x), len(y))
(N_rank, N_accum, N_size_prod) = get_N(N_size)
## marginal densities
marginal_x_density_type = "Uniform"
marginal_x_params_dict = {}
marginal_x_params_dict["location"] = 0
marginal_x_params_dict["scale"] = 1
marginal_x_params_dict["@*1"] = 0
marginal_x_params_dict["@*x"] = 0
marginal_x_params_dict["@*xx"] = 0
marginal_x_params_dict["@*xxx"] = 0
marginal_x_params_dict["@*xxxx"] = 0
marginal_x_params_dict["@*abs(1+x+xx)"] = 0
marginal_x_params_dict["abs(@*1+x+xx)"] = 0
marginal_x_params_dict["abs(1+@*x+xx)"] = 1
marginal_x_params_dict["abs(1+x+@*xx)"] = 0
marginal_x_params_dict["@*sin(1+x+xx)"] = 0
marginal_x_params_dict["sin(@*1+x+xx)"] = 0
marginal_x_params_dict["sin(1+@*x+xx)"] = 1
marginal_x_params_dict["sin(1+x+@*xx)"] = 0
marginal_x_params_dict["@*cos(1+x+xx)"] = 0
marginal_x_params_dict["cos(@*1+x+xx)"] = 0
marginal_x_params_dict["cos(1+@*x+xx)"] = 1
marginal_x_params_dict["cos(1+x+@*xx)"] = 0
marginal_y_density_type = "Uniform"
marginal_y_params_dict = {}
marginal_y_params_dict["location"] = 0
marginal_y_params_dict["scale"] = 1
marginal_y_params_dict["@*1"] = 0
marginal_y_params_dict["@*x"] = 0
marginal_y_params_dict["@*xx"] = 0
marginal_y_params_dict["@*xxx"] = 0
marginal_y_params_dict["@*xxxx"] = 0
marginal_y_params_dict["@*abs(1+x+xx)"] = 0
marginal_y_params_dict["abs(@*1+x+xx)"] = 0
marginal_y_params_dict["abs(1+@*x+xx)"] = 1
marginal_y_params_dict["abs(1+x+@*xx)"] = 0
marginal_y_params_dict["@*sin(1+x+xx)"] = 0
marginal_y_params_dict["sin(@*1+x+xx)"] = 0
marginal_y_params_dict["sin(1+@*x+xx)"] = 1
marginal_y_params_dict["sin(1+x+@*xx)"] = 0
marginal_y_params_dict["@*cos(1+x+xx)"] = 0
marginal_y_params_dict["cos(@*1+x+xx)"] = 0
marginal_y_params_dict["cos(1+@*x+xx)"] = 1
marginal_y_params_dict["cos(1+x+@*xx)"] = 0
marginal_density_ndarray_list = gen_marginal_densities_ndarray_list_for_mindemo(N_rank, N_size, xyz_ndarray_list,
                                                                               [marginal_x_density_type, marginal_y_density_type],
                                                                               [marginal_x_params_dict, marginal_y_params_dict],
                                                                               numerical_precision)
## BEGIN: mindemo params
mindemo_params_dict = {}
mindemo_params_dict["@*1"] = 0
mindemo_params_dict["@*x"] = 0
mindemo_params_dict["@*y"] = 0
mindemo_params_dict["@*xx"] = 0
mindemo_params_dict["@*xy"] = 0
mindemo_params_dict["@*yy"] = 0
mindemo_params_dict["@*xxx"] = 0
mindemo_params_dict["@*xxy"] = 1
mindemo_params_dict["@*xyy"] = 0
mindemo_params_dict["@*yyy"] = 0
mindemo_params_dict["@*xxxx"] = 0
mindemo_params_dict["@*xxxy"] = 0
mindemo_params_dict["@*xxyy"] = 0
mindemo_params_dict["@*xyyy"] = 0
mindemo_params_dict["@*yyyy"] = 0
mindemo_params_dict["@*abs(1+x+y+xx+xy+yy)"] = 0
mindemo_params_dict["abs(@*1+x+y+xx+xy+yy)"] = 0
mindemo_params_dict["abs(1+@*x+y+xx+xy+yy)"] = 1
mindemo_params_dict["abs(1+x+@*y+xx+xy+yy)"] = 1
mindemo_params_dict["abs(1+x+y+@*xx+xy+yy)"] = 0
mindemo_params_dict["abs(1+x+y+xx+@*xy+yy)"] = 0
mindemo_params_dict["abs(1+x+y+xx+xy+@*yy)"] = 0
mindemo_params_dict["@*sin(1+x+y+xx+xy+yy)"] = 0
mindemo_params_dict["sin(@*1+x+y+xx+xy+yy)"] = 0
mindemo_params_dict["sin(1+@*x+y+xx+xy+yy)"] = 0
mindemo_params_dict["sin(1+x+@*y+xx+xy+yy)"] = 0
mindemo_params_dict["sin(1+x+y+@*xx+xy+yy)"] = 3
mindemo_params_dict["sin(1+x+y+xx+@*xy+yy)"] = 0
mindemo_params_dict["sin(1+x+y+xx+xy+@*yy)"] = 3
mindemo_params_dict["@*cos(1+x+y+xx+xy+yy)"] = 0
mindemo_params_dict["cos(@*1+x+y+xx+xy+yy)"] = 0
mindemo_params_dict["cos(1+@*x+y+xx+xy+yy)"] = 0
mindemo_params_dict["cos(1+x+@*y+xx+xy+yy)"] = 0
mindemo_params_dict["cos(1+x+y+@*xx+xy+yy)"] = 3
mindemo_params_dict["cos(1+x+y+xx+@*xy+yy)"] = 0
mindemo_params_dict["cos(1+x+y+xx+xy+@*yy)"] = 3
## END: mindemo params
## optimal transport
(cost_tensor, normalized_cost_tensor, objective_function_value, P_tensor, weighted_cost_tensor,
 u_vec_list, f_vec_list) = gen_density_for_mindemo(xyz_ndarray_list, marginal_density_ndarray_list,
                                                 N_size, N_rank, N_accum, N_size_prod, mindemo_params_dict, numerical_precision)
# cost_array = np.reshape(a=cost_tensor, newshape=N_size, order="F")
# print(cost_array)
# P_array = np.reshape(a=P_tensor, newshape=N_size, order="F")
# print(P_array)
## P_tensor = density
(x_line_chart_fig, x_line_chart_ax) = gen_marginal_density_line_chart(xyz_ndarray_list[0], marginal_density_ndarray_list[0], True)
(y_line_chart_fig, y_line_chart_ax) = gen_marginal_density_line_chart(xyz_ndarray_list[1], marginal_density_ndarray_list[1], True)
(xy_heatmap_fig, xy_heatmap_ax) = gen_heatmap(P_tensor, xyz_ndarray_list, N_size, True)


### WebUI with Gradio

In [ ]:
# !pip install gradio
import gradio as gr

class GradioParameters:
    marginal_x_density_type = None
    marginal_y_density_type = None
    marginal_x_params_dict = None
    marginal_y_params_dict = None
    marginal_params_key_list = None
    mindemo_params_dict = None
    mindemo_params_key_list = None
    xyz_step = None
    xyz_min = None
    xyz_max = None
    xyz_max_plus = None
    x = None
    y = None
    xyz_ndarray_list = None
    N_size = None
    N_rank = None
    N_accum = None
    N_size_prod = None
    marginal_density_ndarray_list = None
    numerical_precision = None
    cost_tensor = None
    normalized_cost_tensor = None
    objective_function_value = None
    P_tensor = None
    weighted_cost_tensor = None
    u_vec_list = None
    f_vec_list = None
    xy_location_parameters_list = None
    xy_scale_parameters_list = None
    xy_heatmap_fig = None
    xy_heatmap_ax = None
    x_line_chart_fig = None
    x_line_chart_ax = None
    y_line_chart_fig = None
    y_line_chart_ax = None

## BEGIN: marginal_x_params_dict (The order is important!!)
GradioParameters.marginal_x_density_type = "uniform"
GradioParameters.marginal_x_params_dict = {}
GradioParameters.marginal_x_params_dict["location"] = 0
GradioParameters.marginal_x_params_dict["scale"] = 1
GradioParameters.marginal_x_params_dict["@*1"] = 0
GradioParameters.marginal_x_params_dict["@*x"] = 1
GradioParameters.marginal_x_params_dict["@*xx"] = 0
GradioParameters.marginal_x_params_dict["@*xxx"] = 0
GradioParameters.marginal_x_params_dict["@*xxxx"] = 0
GradioParameters.marginal_x_params_dict["@*abs(1+x+xx)"] = 0
GradioParameters.marginal_x_params_dict["abs(@*1+x+xx)"] = 0
GradioParameters.marginal_x_params_dict["abs(1+@*x+xx)"] = 1
GradioParameters.marginal_x_params_dict["abs(1+x+@*xx)"] = 0
GradioParameters.marginal_x_params_dict["@*sin(1+x+xx)"] = 0
GradioParameters.marginal_x_params_dict["sin(@*1+x+xx)"] = 0
GradioParameters.marginal_x_params_dict["sin(1+@*x+xx)"] = 1
GradioParameters.marginal_x_params_dict["sin(1+x+@*xx)"] = 0
GradioParameters.marginal_x_params_dict["@*cos(1+x+xx)"] = 0
GradioParameters.marginal_x_params_dict["cos(@*1+x+xx)"] = 0
GradioParameters.marginal_x_params_dict["cos(1+@*x+xx)"] = 1
GradioParameters.marginal_x_params_dict["cos(1+x+@*xx)"] = 0
## END: marginal_x_params_dict
## BEGIN: marginal_y_params_dict (The order is important!!)
GradioParameters.marginal_y_density_type = "uniform"
GradioParameters.marginal_y_params_dict = {}
GradioParameters.marginal_y_params_dict["location"] = 0
GradioParameters.marginal_y_params_dict["scale"] = 1
GradioParameters.marginal_y_params_dict["@*1"] = 0
GradioParameters.marginal_y_params_dict["@*x"] = 1
GradioParameters.marginal_y_params_dict["@*xx"] = 0
GradioParameters.marginal_y_params_dict["@*xxx"] = 0
GradioParameters.marginal_y_params_dict["@*xxxx"] = 0
GradioParameters.marginal_y_params_dict["@*abs(1+x+xx)"] = 0
GradioParameters.marginal_y_params_dict["abs(@*1+x+xx)"] = 0
GradioParameters.marginal_y_params_dict["abs(1+@*x+xx)"] = 1
GradioParameters.marginal_y_params_dict["abs(1+x+@*xx)"] = 0
GradioParameters.marginal_y_params_dict["@*sin(1+x+xx)"] = 0
GradioParameters.marginal_y_params_dict["sin(@*1+x+xx)"] = 0
GradioParameters.marginal_y_params_dict["sin(1+@*x+xx)"] = 1
GradioParameters.marginal_y_params_dict["sin(1+x+@*xx)"] = 0
GradioParameters.marginal_y_params_dict["@*cos(1+x+xx)"] = 0
GradioParameters.marginal_y_params_dict["cos(@*1+x+xx)"] = 0
GradioParameters.marginal_y_params_dict["cos(1+@*x+xx)"] = 1
GradioParameters.marginal_y_params_dict["cos(1+x+@*xx)"] = 0
## END: marginal_y_params_dict
GradioParameters.marginal_params_keys_list = ["location", "scale", ## (The order is important!!)
                            "@*1", "@*x", "@*xx", "@*xxx", "@*xxxx",
                            "@*abs(1+x+xx)", "abs(@*1+x+xx)", "abs(1+@*x+xx)", "abs(1+x+@*xx)",
                            "@*sin(1+x+xx)", "sin(@*1+x+xx)", "sin(1+@*x+xx)", "sin(1+x+@*xx)",
                            "@*cos(1+x+xx)", "cos(@*1+x+xx)", "cos(1+@*x+xx)", "cos(1+x+@*xx)"]
## BEGIN: mindemo_params_dict (The order is important!!)
GradioParameters.mindemo_params_dict = {}
GradioParameters.mindemo_params_dict["@*1"] = 0
GradioParameters.mindemo_params_dict["@*x"] = 0
GradioParameters.mindemo_params_dict["@*y"] = 0
GradioParameters.mindemo_params_dict["@*xx"] = 0
GradioParameters.mindemo_params_dict["@*xy"] = 1
GradioParameters.mindemo_params_dict["@*yy"] = 0
GradioParameters.mindemo_params_dict["@*xxx"] = 0
GradioParameters.mindemo_params_dict["@*xxy"] = 0
GradioParameters.mindemo_params_dict["@*xyy"] = 0
GradioParameters.mindemo_params_dict["@*yyy"] = 0
GradioParameters.mindemo_params_dict["@*xxxx"] = 0
GradioParameters.mindemo_params_dict["@*xxxy"] = 0
GradioParameters.mindemo_params_dict["@*xxyy"] = 0
GradioParameters.mindemo_params_dict["@*xyyy"] = 0
GradioParameters.mindemo_params_dict["@*yyyy"] = 0
GradioParameters.mindemo_params_dict["@*abs(1+x+y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["abs(@*1+x+y+xx+xy+yy)"] = 1
GradioParameters.mindemo_params_dict["abs(1+@*x+y+xx+xy+yy)"] = 1
GradioParameters.mindemo_params_dict["abs(1+x+@*y+xx+xy+yy)"] = 1
GradioParameters.mindemo_params_dict["abs(1+x+y+@*xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["abs(1+x+y+xx+@*xy+yy)"] = 0
GradioParameters.mindemo_params_dict["abs(1+x+y+xx+xy+@*yy)"] = 0
GradioParameters.mindemo_params_dict["@*sin(1+x+y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["sin(@*1+x+y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["sin(1+@*x+y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["sin(1+x+@*y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["sin(1+x+y+@*xx+xy+yy)"] = 3
GradioParameters.mindemo_params_dict["sin(1+x+y+xx+@*xy+yy)"] = 0
GradioParameters.mindemo_params_dict["sin(1+x+y+xx+xy+@*yy)"] = 3
GradioParameters.mindemo_params_dict["@*cos(1+x+y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["cos(@*1+x+y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["cos(1+@*x+y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["cos(1+x+@*y+xx+xy+yy)"] = 0
GradioParameters.mindemo_params_dict["cos(1+x+y+@*xx+xy+yy)"] = 3
GradioParameters.mindemo_params_dict["cos(1+x+y+xx+@*xy+yy)"] = 0
GradioParameters.mindemo_params_dict["cos(1+x+y+xx+xy+@*yy)"] = 3
GradioParameters.mindemo_params_keys_list = ["@*1", ## (The order is important!!)
                                             "@*x","@*y",
                                             "@*xx","@*xy","@*yy",
                                             "@*xxx","@*xxy","@*xyy","@*yyy",
                                             "@*xxxx","@*xxxy","@*xxyy","@*xyyy","@*yyyy",
                                             "@*abs(1+x+y+xx+xy+yy)","abs(@*1+x+y+xx+xy+yy)","abs(1+@*x+y+xx+xy+yy)","abs(1+x+@*y+xx+xy+yy)",
                                             "abs(1+x+y+@*xx+xy+yy)","abs(1+x+y+xx+@*xy+yy)","abs(1+x+y+xx+xy+@*yy)",
                                             "@*sin(1+x+y+xx+xy+yy)","sin(@*1+x+y+xx+xy+yy)","sin(1+@*x+y+xx+xy+yy)","sin(1+x+@*y+xx+xy+yy)",
                                             "sin(1+x+y+@*xx+xy+yy)","sin(1+x+y+xx+@*xy+yy)","sin(1+x+y+xx+xy+@*yy)",
                                             "@*cos(1+x+y+xx+xy+yy)","cos(@*1+x+y+xx+xy+yy)","cos(1+@*x+y+xx+xy+yy)","cos(1+x+@*y+xx+xy+yy)",
                                             "cos(1+x+y+@*xx+xy+yy)","cos(1+x+y+xx+@*xy+yy)","cos(1+x+y+xx+xy+@*yy)"]
## END: mindemo_params_dict
GradioParameters.xyz_step = 0.05
GradioParameters.xyz_min = -1
GradioParameters.xyz_max = 1
GradioParameters.xyz_max_plus = GradioParameters.xyz_max + GradioParameters.xyz_step
x = np.arange(GradioParameters.xyz_min, GradioParameters.xyz_max_plus, GradioParameters.xyz_step)
y = np.arange(GradioParameters.xyz_min, GradioParameters.xyz_max_plus, GradioParameters.xyz_step)
GradioParameters.xyz_ndarray_list = [x, y]
GradioParameters.N_size = (len(x), len(y))
(GradioParameters.N_rank, GradioParameters.N_accum, GradioParameters.N_size_prod) = get_N(GradioParameters.N_size)
GradioParameters.numerical_precision = 2e-8
GradioParameters.marginal_density_ndarray_list = gen_marginal_densities_ndarray_list_for_mindemo(GradioParameters.N_rank, GradioParameters.N_size,
                                                                                    GradioParameters.xyz_ndarray_list,
                                                                                    [GradioParameters.marginal_x_density_type, GradioParameters.marginal_y_density_type],
                                                                                    [GradioParameters.marginal_x_params_dict, GradioParameters.marginal_y_params_dict])
(GradioParameters.cost_tensor, GradioParameters.normalized_cost_tensor, GradioParameters.objective_function_value, GradioParameters.P_tensor, GradioParameters.weighted_cost_tensor,
 GradioParameters.u_vec_list, GradioParameters.f_vec_list) = gen_density_for_mindemo(GradioParameters.xyz_ndarray_list, GradioParameters.marginal_density_ndarray_list,
                                                 GradioParameters.N_size, GradioParameters.N_rank, GradioParameters.N_accum, GradioParameters.N_size_prod,
                                                 GradioParameters.mindemo_params_dict, GradioParameters.numerical_precision)
(GradioParameters.x_line_chart_fig, GradioParameters.x_line_chart_ax) = gen_marginal_density_line_chart(GradioParameters.xyz_ndarray_list[0], GradioParameters.marginal_density_ndarray_list[0])
(GradioParameters.y_line_chart_fig, GradioParameters.y_line_chart_ax) = gen_marginal_density_line_chart(GradioParameters.xyz_ndarray_list[1], GradioParameters.marginal_density_ndarray_list[1])
(GradioParameters.xy_heatmap_fig, GradioParameters.xy_heatmap_ax) = gen_heatmap(GradioParameters.P_tensor, GradioParameters.xyz_ndarray_list, GradioParameters.N_size)

def gen_heatmap_after_parameter_update_in_gradio():
    (GradioParameters.cost_tensor, GradioParameters.normalized_cost_tensor, GradioParameters.objective_function_value, GradioParameters.P_tensor, GradioParameters.weighted_cost_tensor,
     GradioParameters.u_vec_list, GradioParameters.f_vec_list) = gen_density_for_mindemo(GradioParameters.xyz_ndarray_list, GradioParameters.marginal_density_ndarray_list,
                                                 GradioParameters.N_size, GradioParameters.N_rank, GradioParameters.N_accum, GradioParameters.N_size_prod,
                                                 GradioParameters.mindemo_params_dict, GradioParameters.numerical_precision)
    (GradioParameters.xy_heatmap_fig, GradioParameters.xy_heatmap_ax) = gen_heatmap(GradioParameters.P_tensor, GradioParameters.xyz_ndarray_list, GradioParameters.N_size)
    return GradioParameters.xy_heatmap_fig
def gen_heatmap_and_line_chart_after_parameter_update_in_gradio():
    GradioParameters.marginal_density_ndarray_list = gen_marginal_densities_ndarray_list_for_mindemo(GradioParameters.N_rank, GradioParameters.N_size,
                                                                                    GradioParameters.xyz_ndarray_list,
                                                                                    [GradioParameters.marginal_x_density_type, GradioParameters.marginal_y_density_type],
                                                                                    [GradioParameters.marginal_x_params_dict, GradioParameters.marginal_y_params_dict])
    GradioParameters.xy_heatmap_fig = gen_heatmap_after_parameter_update_in_gradio()
    (GradioParameters.x_line_chart_fig, GradioParameters.x_line_chart_ax) = gen_marginal_density_line_chart(GradioParameters.xyz_ndarray_list[0], GradioParameters.marginal_density_ndarray_list[0])
    (GradioParameters.y_line_chart_fig, GradioParameters.y_line_chart_ax) = gen_marginal_density_line_chart(GradioParameters.xyz_ndarray_list[1], GradioParameters.marginal_density_ndarray_list[1])
    return (GradioParameters.xy_heatmap_fig, GradioParameters.x_line_chart_fig, GradioParameters.y_line_chart_fig)
def step_size_change(param):
    GradioParameters.xyz_step = param
    GradioParameters.xyz_max_plus = GradioParameters.xyz_max + GradioParameters.xyz_step
    x = np.arange(GradioParameters.xyz_min, GradioParameters.xyz_max_plus, GradioParameters.xyz_step)
    y = np.arange(GradioParameters.xyz_min, GradioParameters.xyz_max_plus, GradioParameters.xyz_step)
    GradioParameters.xyz_ndarray_list = [x, y]
    GradioParameters.N_size = (len(x), len(y))
    (GradioParameters.N_rank, GradioParameters.N_accum, GradioParameters.N_size_prod) = get_N(GradioParameters.N_size)
    return gen_heatmap_and_line_chart_after_parameter_update_in_gradio()

def marginal_dist_change(marginal_x_density_type, ## (The order is important!!)
                         marginal_x_param_location, marginal_x_param_scale,
                         marginal_x_param_1, marginal_x_param_x, marginal_x_param_xx, marginal_x_param_xxx, marginal_x_param_xxxx,
                         marginal_x_param_ABS1, marginal_x_param_ABS2, marginal_x_param_ABS3, marginal_x_param_ABS4,
                         marginal_x_param_SIN1, marginal_x_param_SIN2, marginal_x_param_SIN3, marginal_x_param_SIN4,
                         marginal_x_param_COS1, marginal_x_param_COS2, marginal_x_param_COS3, marginal_x_param_COS4,
                         marginal_y_density_type,
                         marginal_y_param_location, marginal_y_param_scale,
                         marginal_y_param_1, marginal_y_param_x, marginal_y_param_xx, marginal_y_param_xxx, marginal_y_param_xxxx,
                         marginal_y_param_ABS1, marginal_y_param_ABS2, marginal_y_param_ABS3, marginal_y_param_ABS4,
                         marginal_y_param_SIN1, marginal_y_param_SIN2, marginal_y_param_SIN3, marginal_y_param_SIN4,
                         marginal_y_param_COS1, marginal_y_param_COS2, marginal_y_param_COS3, marginal_y_param_COS4):
    marginal_x_values_list = [marginal_x_param_location, marginal_x_param_scale, ## (The order is important!!)                
                    marginal_x_param_1, marginal_x_param_x, marginal_x_param_xx, marginal_x_param_xxx, marginal_x_param_xxxx,
                    marginal_x_param_ABS1, marginal_x_param_ABS2, marginal_x_param_ABS3, marginal_x_param_ABS4,
                    marginal_x_param_SIN1, marginal_x_param_SIN2, marginal_x_param_SIN3, marginal_x_param_SIN4,
                    marginal_x_param_COS1, marginal_x_param_COS2, marginal_x_param_COS3, marginal_x_param_COS4]
    marginal_y_values_list = [marginal_y_param_location, marginal_y_param_scale, ## (The order is important!!)
                    marginal_y_param_1, marginal_y_param_x, marginal_y_param_xx, marginal_y_param_xxx, marginal_y_param_xxxx,
                    marginal_y_param_ABS1, marginal_y_param_ABS2, marginal_y_param_ABS3, marginal_y_param_ABS4,
                    marginal_y_param_SIN1, marginal_y_param_SIN2, marginal_y_param_SIN3, marginal_y_param_SIN4,
                    marginal_y_param_COS1, marginal_y_param_COS2, marginal_y_param_COS3, marginal_y_param_COS4]
    GradioParameters.marginal_x_density_type = marginal_x_density_type
    GradioParameters.marginal_y_density_type = marginal_y_density_type
    for i, key in enumerate(GradioParameters.marginal_params_keys_list):
        # print( str(key) + ": " + str(marginal_x_values_list[i]))
        # print( str(key) + ": " + str(marginal_y_values_list[i]))
        GradioParameters.marginal_x_params_dict[key] = marginal_x_values_list[i]
        GradioParameters.marginal_y_params_dict[key] = marginal_y_values_list[i]
    return gen_heatmap_and_line_chart_after_parameter_update_in_gradio()

def xy_location_scale_change(x_location, x_scale, y_location, y_scale):
    GradioParameters.xy_location_parameters_list[0] = x_location 
    GradioParameters.xy_scale_parameters_list[0] = x_scale
    GradioParameters.xy_location_parameters_list[1] = y_location
    GradioParameters.xy_scale_parameters_list[1] = y_scale
    return gen_heatmap_and_line_chart_after_parameter_update_in_gradio()
def mindemo_param_change(mindemo_param_1, ##  (The order is important!!)
                         mindemo_param_x, mindemo_param_y,
                         mindemo_param_xx, mindemo_param_xy, mindemo_param_yy,
                         mindemo_param_xxx, mindemo_param_xxy, mindemo_param_xyy, mindemo_param_yyy,
                         mindemo_param_xxxx, mindemo_param_xxxy, mindemo_param_xxyy, mindemo_param_xyyy, mindemo_param_yyyy,
                         mindemo_param_ABS1, mindemo_param_ABS2, mindemo_param_ABS3, mindemo_param_ABS4, mindemo_param_ABS5, mindemo_param_ABS6, mindemo_param_ABS7,
                         mindemo_param_SIN1, mindemo_param_SIN2, mindemo_param_SIN3, mindemo_param_SIN4, mindemo_param_SIN5, mindemo_param_SIN6, mindemo_param_SIN7,
                         mindemo_param_COS1, mindemo_param_COS2, mindemo_param_COS3, mindemo_param_COS4, mindemo_param_COS5, mindemo_param_COS6, mindemo_param_COS7):
    values_list = [mindemo_param_1, ##  (The order is important!!)
                   mindemo_param_x, mindemo_param_y,
                   mindemo_param_xx, mindemo_param_xy, mindemo_param_yy,
                   mindemo_param_xxx, mindemo_param_xxy, mindemo_param_xyy, mindemo_param_yyy,
                   mindemo_param_xxxx, mindemo_param_xxxy, mindemo_param_xxyy, mindemo_param_xyyy, mindemo_param_yyyy,
                   mindemo_param_ABS1, mindemo_param_ABS2, mindemo_param_ABS3, mindemo_param_ABS4, mindemo_param_ABS5, mindemo_param_ABS6, mindemo_param_ABS7,
                   mindemo_param_SIN1, mindemo_param_SIN2, mindemo_param_SIN3, mindemo_param_SIN4, mindemo_param_SIN5, mindemo_param_SIN6, mindemo_param_SIN7,
                   mindemo_param_COS1, mindemo_param_COS2, mindemo_param_COS3, mindemo_param_COS4, mindemo_param_COS5, mindemo_param_COS6, mindemo_param_COS7]
    for i, key in enumerate(GradioParameters.mindemo_params_keys_list):
        print( str(key) + ": " + str(values_list[i]))
        GradioParameters.mindemo_params_dict[key] = values_list[i]
    return gen_heatmap_after_parameter_update_in_gradio()

equation_script_str = ""
with gr.Blocks(title="Mindemo") as demo:
    with gr.Row():
        with gr.Column():
            with gr.Group():
                gr.Markdown(value="Settings for marginal distributions")
                with gr.Tab(label="x"):
                    marginal_x_density_type_dropdown = gr.Dropdown(choices=["Uniform", "Normal", "Exp"], value="Uniform",
                                        label="Marginal distribution", info="Choose marginal distribution of x", interactive=True)
                    with gr.Accordion(label="Advanced settings for marginal distribution of x", open=False):
                        with gr.Group():
                            marginal_x_param_location_slider = gr.Slider(minimum=-1.0, maximum=1.0, step=0.1, value=0.0,
                                        label="Location of x (for Uniform, Normal)", info="Choose between -1.0 and 1.0", interactive=True)
                            marginal_x_param_scale_slider = gr.Slider(minimum=0.1, maximum=10.0, step=0.1, value=1.0,
                                        label="Scale of x (for Uniform, Normal)", info="Choose between 0.1 and 10.0", interactive=True)
                        with gr.Group():
                            marginal_x_param_exp_str_a = "$$ a_1 + a_2 x + a_3 x^2 + a_4 x^3 + a_5 x^4 + \\cdots $$"
                            marginal_x_param_exp_str_b = "$$ \\cdots + b_1\\left|b_2 + b_3x + b_4x^2\\right| + \\cdots$$"
                            marginal_x_param_exp_str_c = "$$ \\cdots + c_1\\sin(2\\pi(c_2 + c_3x + c_4x^2)) + \\cdots + \\cdots$$"
                            marginal_x_param_exp_str_d = "$$ \\cdots + d_1\\cos(2\\pi(d_2 + d_3x + d_4x^2)) $$"
                            gr.Markdown(value=equation_script_str + marginal_x_param_exp_str_a)
                            marginal_x_param_1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_x_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="a2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_xx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_xxx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_xxxx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a5 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            gr.Markdown(value=equation_script_str + marginal_x_param_exp_str_b)
                            marginal_x_param_ABS1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="b1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_ABS2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="b2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_ABS3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="b3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_ABS4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="b4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            gr.Markdown(value=equation_script_str + marginal_x_param_exp_str_c)
                            marginal_x_param_SIN1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="c1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_SIN2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="c2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_SIN3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="c3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_SIN4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="c4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            gr.Markdown(value=equation_script_str + marginal_x_param_exp_str_d)
                            marginal_x_param_COS1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="d1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_COS2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="d2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_COS3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="d3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_x_param_COS4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="d4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                with gr.Tab(label="y"):
                    marginal_y_density_type_dropdown = gr.Dropdown(choices=["Uniform", "Normal", "Exp"], value="Uniform",
                                        label="Marginal distribution", info="Choose marginal distribution of y", interactive=True)
                    with gr.Accordion(label="Advanced settings for marginal distribution of y", open=False):
                        with gr.Group():
                            marginal_y_param_location_slider = gr.Slider(minimum=-1.0, maximum=1.0, step=0.1, value=0.0,
                                        label="Location of y (for Uniform, Normal)", info="Choose between -1.0 and 1.0", interactive=True)
                            marginal_y_param_scale_slider = gr.Slider(minimum=0.1, maximum=10.0, step=0.1, value=1.0,
                                    label="Scale of y (for Uniform, Normal)", info="Choose between 0.1 and 10.0", interactive=True)
                        with gr.Group():
                            marginal_y_param_exp_str_a = "$$ a_1 + a_2 y + a_3 y^2 + a_4 y^3 + a_5 y^4 + \\cdots $$"
                            marginal_y_param_exp_str_b = "$$ \\cdots + b_1\\left|b_2 + b_3y + b_4y^2\\right| + \\cdots$$"
                            marginal_y_param_exp_str_c = "$$ \\cdots + c_1\\sin(2\\pi(c_2 + c_3y + c_4y^2)) + \\cdots + \\cdots$$"
                            marginal_y_param_exp_str_d = "$$ \\cdots + d_1\\cos(2\\pi(d_2 + d_3y + d_4y^2)) $$"
                            gr.Markdown(value=equation_script_str + marginal_y_param_exp_str_a)
                            marginal_y_param_1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_x_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="a2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_xx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_xxx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_xxxx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="a5 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            gr.Markdown(value=equation_script_str + marginal_y_param_exp_str_b)
                            marginal_y_param_ABS1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="b1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_ABS2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="b2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_ABS3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="b3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_ABS4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="b4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            gr.Markdown(value=equation_script_str + marginal_y_param_exp_str_c)
                            marginal_y_param_SIN1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="c1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_SIN2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="c2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_SIN3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="c3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_SIN4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="c4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            gr.Markdown(value=equation_script_str + marginal_y_param_exp_str_d)
                            marginal_y_param_COS1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="d1 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_COS2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="d2 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_COS3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0,
                                        label="d3 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
                            marginal_y_param_COS4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0,
                                        label="d4 (for Exp)", info="Choose between -10.0 and 10.0", interactive=True)
            with gr.Group():
                gr.Markdown(value="Settings for Mindemo")
                with gr.Tab(label="Polynomial"):
                    mindemo_param_polynomial_str_1 = "$$ a_1 + a_2 x + a_3 y + a_4 x^2 + a_5 xy + a_6 y^2 + \\cdots$$"
                    mindemo_param_polynomial_str_2 = "$$ \\cdots + a_7 x^3 + a_8 x^2y + a_9 xy^2 + a_{10}y^3 + \\cdots$$"
                    mindemo_param_polynomial_str_3 = "$$ \\cdots + a_{11} x^4 + a_{12} x^3y + a_{13} x^2y^2 + a_{14} xy^3 + a_{15} y^4 + \\cdots$$"
                    gr.Markdown(value=equation_script_str + mindemo_param_polynomial_str_1)
                    mindemo_param_1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a1", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_x_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a2", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_y_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a3", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_xx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a4", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_xy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0, 
                                        label="a5", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_yy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a6", info="Choose between -10.0 and 10.0", interactive=True)
                    gr.Markdown(value=equation_script_str + mindemo_param_polynomial_str_2)
                    mindemo_param_xxx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a7", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_xxy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a8", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_xyy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a9", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_yyy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a10", info="Choose between -10.0 and 10.0", interactive=True)
                    gr.Markdown(value=equation_script_str + mindemo_param_polynomial_str_3)
                    mindemo_param_xxxx_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a11", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_xxxy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a12", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_xxyy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a13", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_xyyy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a14", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_yyyy_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                        label="a15", info="Choose between -10.0 and 10.0", interactive=True)
                with gr.Tab(label="Abs"):
                    mindemo_param_abs_str = "$$ \\cdots + b_1\\left|b_2 + b_3x + b_4y + b_5x^2 + b_6xy + b_7y^2\\right| + \\cdots $$"
                    gr.Markdown(value=equation_script_str + mindemo_param_abs_str)
                    mindemo_param_ABS1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="b1", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_ABS2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="b2", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_ABS3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0, 
                                    label="b3", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_ABS4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=1.0, 
                                    label="b4", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_ABS5_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="b5", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_ABS6_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="b6", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_ABS7_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="b7", info="Choose between -10.0 and 10.0", interactive=True)
                with gr.Tab(label="Sin"):
                    mindemo_param_sin_str = "$$ \\cdots + c_1\\sin(2\\pi(c_2 + c_3x + c_4y + c_5x^2 + c_6xy + c_7y^2)) + \\cdots $$"
                    gr.Markdown(value=equation_script_str + mindemo_param_sin_str)
                    mindemo_param_SIN1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="c1", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_SIN2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="c2", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_SIN3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="c3", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_SIN4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="c4", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_SIN5_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=3.0, 
                                    label="c5", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_SIN6_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="c6", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_SIN7_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=3.0, 
                                    label="c7", info="Choose between -10.0 and 10.0", interactive=True)
                with gr.Tab(label="Cos"):
                    mindemo_param_cos_str = "$$ \\cdots + d_1\\cos(2\\pi(d_2 + d_3x + d_4y + d_5x^2 + d_6xy + d_7y^2)) $$"
                    gr.Markdown(value=equation_script_str + mindemo_param_cos_str)
                    mindemo_param_COS1_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="d1", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_COS2_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="d2", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_COS3_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="d3", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_COS4_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="d4", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_COS5_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=3.0, 
                                    label="d5", info="Choose between -10.0 and 10.0", interactive=True)
                    mindemo_param_COS6_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=0.0, 
                                    label="d6", info="Choose between -10.0 and 10.0", interactive=True) 
                    mindemo_param_COS7_slider = gr.Slider(minimum=-10.0, maximum=10.0, step=0.1, value=3.0, 
                                    label="d7", info="Choose between -10.0 and 10.0", interactive=True)            
        with gr.Column():
            with gr.Row():
                step_size_slider = gr.Slider(minimum=0.005, maximum=0.5, step=0.005, value=0.05, 
                                label="Step size", info="Choose between 0.005 and 0.5", interactive=True)
            with gr.Row():
                x_line_chart_image = gr.Plot(label="Marginal density of x", value=GradioParameters.x_line_chart_fig)
                y_line_chart_image = gr.Plot(label="Marginal density of y", value=GradioParameters.y_line_chart_fig)
            with gr.Row():
                xy_heatmap_image = gr.Plot(label="Joint density of x and y", value=GradioParameters.xy_heatmap_fig)
    marginal_x_params_sliders_list = [marginal_x_param_location_slider, marginal_x_param_scale_slider, ##  (The order is important!!)
                                   marginal_x_param_1_slider, marginal_x_param_x_slider, marginal_x_param_xx_slider, marginal_x_param_xxx_slider, marginal_x_param_xxxx_slider, 
                                   marginal_x_param_ABS1_slider, marginal_x_param_ABS2_slider, marginal_x_param_ABS3_slider, marginal_x_param_ABS4_slider,
                                   marginal_x_param_SIN1_slider, marginal_x_param_SIN2_slider, marginal_x_param_SIN3_slider, marginal_x_param_SIN4_slider,
                                   marginal_x_param_COS1_slider, marginal_x_param_COS2_slider, marginal_x_param_COS3_slider, marginal_x_param_COS4_slider]
    marginal_y_params_sliders_list = [marginal_y_param_location_slider, marginal_y_param_scale_slider, ##  (The order is important!!)
                                   marginal_y_param_1_slider, marginal_y_param_x_slider, marginal_y_param_xx_slider, marginal_y_param_xxx_slider, marginal_y_param_xxxx_slider, 
                                   marginal_y_param_ABS1_slider, marginal_y_param_ABS2_slider, marginal_y_param_ABS3_slider, marginal_y_param_ABS4_slider,
                                   marginal_y_param_SIN1_slider, marginal_y_param_SIN2_slider, marginal_y_param_SIN3_slider, marginal_y_param_SIN4_slider,
                                   marginal_y_param_COS1_slider, marginal_y_param_COS2_slider, marginal_y_param_COS3_slider, marginal_y_param_COS4_slider]
    marginal_dist_inputs_list = [marginal_x_density_type_dropdown]+marginal_x_params_sliders_list+[marginal_y_density_type_dropdown]+marginal_y_params_sliders_list
    marginal_dist_outputs_list = [xy_heatmap_image, x_line_chart_image, y_line_chart_image]
    marginal_x_density_type_dropdown.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_location_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_scale_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_x_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_xx_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_xxx_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_xxxx_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_ABS1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_ABS2_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_ABS3_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_ABS4_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_SIN1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_SIN2_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_SIN3_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_SIN4_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_COS1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_COS2_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_COS3_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_x_param_COS4_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_density_type_dropdown.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_location_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_scale_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_x_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_xx_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_xxx_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_xxxx_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_ABS1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_ABS2_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_ABS3_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_ABS4_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_SIN1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_SIN2_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_SIN3_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_SIN4_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_COS1_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_COS2_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_COS3_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    marginal_y_param_COS4_slider.change(fn=marginal_dist_change, inputs=marginal_dist_inputs_list, outputs=marginal_dist_outputs_list)
    mindemo_params_sliders_list = [mindemo_param_1_slider, ##  (The order is important!!)
                                   mindemo_param_x_slider, mindemo_param_y_slider,
                                   mindemo_param_xx_slider, mindemo_param_xy_slider, mindemo_param_yy_slider,
                                   mindemo_param_xxx_slider, mindemo_param_xxy_slider, mindemo_param_xyy_slider, mindemo_param_yyy_slider,
                                   mindemo_param_xxxx_slider, mindemo_param_xxxy_slider, mindemo_param_xxyy_slider, mindemo_param_xyyy_slider, mindemo_param_yyyy_slider,
                                   mindemo_param_ABS1_slider, mindemo_param_ABS2_slider, mindemo_param_ABS3_slider, mindemo_param_ABS4_slider, mindemo_param_ABS5_slider, mindemo_param_ABS6_slider, mindemo_param_ABS7_slider,
                                   mindemo_param_SIN1_slider, mindemo_param_SIN2_slider, mindemo_param_SIN3_slider, mindemo_param_SIN4_slider, mindemo_param_SIN5_slider, mindemo_param_SIN6_slider, mindemo_param_SIN7_slider,
                                   mindemo_param_COS1_slider, mindemo_param_COS2_slider, mindemo_param_COS3_slider, mindemo_param_COS4_slider, mindemo_param_COS5_slider, mindemo_param_COS6_slider, mindemo_param_COS7_slider]
    mindemo_param_1_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_x_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_y_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xx_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_yy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xxx_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xxy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xyy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_yyy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xxxx_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xxxy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xxyy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_xyyy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_yyyy_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_ABS1_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_ABS2_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_ABS3_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_ABS4_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_ABS5_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_ABS6_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_ABS7_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_SIN1_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_SIN2_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_SIN3_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_SIN4_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_SIN5_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_SIN6_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_SIN7_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_COS1_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_COS2_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_COS3_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_COS4_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_COS5_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_COS6_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    mindemo_param_COS7_slider.change(fn=mindemo_param_change, inputs=mindemo_params_sliders_list, outputs=[xy_heatmap_image])
    step_size_slider.change(fn=step_size_change, inputs=[step_size_slider], outputs=[xy_heatmap_image, x_line_chart_image, y_line_chart_image])

demo.css = "footer {visibility: hidden}"
demo.launch(inbrowser=True, show_api=False)
